In [1]:
library(tidyverse)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "cohort.sql"     "data.sql"       "population.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic", user = "lcp_db_user")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "cohort.sql")) %>%
    make_view_sql("cohort", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from cohort limit 10;
"
dbGetQuery(con, sql)

icustay_id,subject_id,hadm_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,echo
200003,27513,163557,SICU,2199-08-02 19:50:04,2199-08-08 17:09:18,1,48.29627,1,2199-08-03,1
200014,9514,127229,SICU,2105-02-16 23:16:48,2105-02-18 16:53:29,1,84.73004,1,2105-02-18,1
200030,14987,165426,MICU,2150-11-13 14:08:02,2150-11-19 17:51:59,1,54.19308,1,2150-11-16,1
200033,56369,198650,SICU,2198-08-07 17:56:17,2198-08-21 14:59:18,1,67.14657,1,NA,0
200036,9960,159243,MICU,2181-08-26 16:38:25,2181-08-29 16:28:48,1,74.93578,1,NA,0
200075,67800,132255,MICU,2159-09-23 00:13:20,2159-09-25 01:55:17,1,83.64320,1,2159-09-25,1
200087,61871,124231,MICU,2196-08-30 11:19:49,2196-09-03 17:40:09,1,64.32851,1,2196-08-31,1
200102,11228,167826,SICU,2193-06-19 01:15:31,2193-06-27 22:25:20,1,46.43237,1,2193-06-19,1
200131,47250,187834,MICU,2176-10-30 12:05:18,2176-11-07 17:34:37,1,69.31159,1,NA,0
200140,27874,193876,SICU,2197-01-18 05:39:31,2197-02-07 15:00:25,1,68.02678,1,2197-01-18,1


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE